In [10]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import time
from collections import Counter
import pickle
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager, rc  
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)

#(빈)크롬창 열기
driver = webdriver.Chrome(ChromeDriverManager().install())

#네이버 지식인: 프로미스나인
driver.get("https://search.naver.com/search.naver?where=kin&sm=tab_jum&query=%EB%B9%84%ED%88%AC%EB%B9%84%EC%82%AC%EC%A7%84")
driver.maximize_window()
time.sleep(3)

In [11]:
url_list=[]

for i in range(1, 101):
    page_url=f'https://search.naver.com/search.naver?where=kin&kin_display=10&qt=&title=0&&answer=0&grade=0&choice=0&sec=0&nso=so%3A-1%2Ca%3A%2Cp%3Aall&query=%EB%B9%84%ED%88%AC%EB%B9%84%EC%82%AC%EC%A7%84&c_id=&c_name=&sm=tab_pge&kin_start={(i-1)*10+1}&kin_age=0'
    driver.get(page_url)
    time.sleep(1)
    

    #객체화
    soup=bs(driver.page_source, 'html.parser')
    url=soup.select('div.question_group>a')
    
    for j in url:
        print(j['href'])
        url_list.append(j['href'])
    time.sleep(1)

https://kin.naver.com/qna/detail.naver?d1id=3&dirId=3021401&docId=433533087&qb=67mE7Yis67mE7IKs7KeE&enc=utf8&section=kin.ext&rank=1&search_sort=0&spq=0
https://kin.naver.com/qna/detail.naver?d1id=5&dirId=50401&docId=429496516&qb=67mE7Yis67mE7IKs7KeE&enc=utf8&section=kin.ext&rank=2&search_sort=0&spq=0
https://kin.naver.com/qna/detail.naver?d1id=3&dirId=30215&docId=433373934&qb=67mE7Yis67mE7IKs7KeE&enc=utf8&section=kin.ext&rank=3&search_sort=0&spq=0
https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80701&docId=433407153&qb=67mE7Yis67mE7IKs7KeE&enc=utf8&section=kin.ext&rank=4&search_sort=0&spq=0
https://kin.naver.com/qna/detail.naver?d1id=3&dirId=302140101&docId=375821081&qb=67mE7Yis67mE7IKs7KeE&enc=utf8&section=kin.ext&rank=5&search_sort=0&spq=0
https://kin.naver.com/qna/detail.naver?d1id=3&dirId=31402&docId=357128860&qb=67mE7Yis67mE7IKs7KeE&enc=utf8&section=kin.ext&rank=6&search_sort=0&spq=0
https://kin.naver.com/qna/detail.naver?d1id=3&dirId=31401&docId=274448036&qb=67mE7Yis67mE7IKs7

In [12]:
len(url_list)

21050

In [13]:
all_contents_list=[]
for i in url_list:
    driver.get(i)
    #제목이랑 내용 함께 추출하고 싶을 때                                   
    all_contents=driver.find_elements_by_css_selector('#content > div.question-content > div')
    for j in all_contents:                              
        all_contents_list.append(j.text)

    
    

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.122)


In [ ]:

#형태소 분석기를 통해 텍스트 정제하기
from konlpy.tag import Okt
okt=Okt()
import re

#필요없는 표시들 지우기
result_list =[]
for i in all_contents_list:
    result = i.replace("\n", "")
    result = result.replace('질문', '')
    result = result.replace('?', '')
    result = result.replace('나눈', '')
    result = result.replace('내공', '')
    result = result.replace('[', '')
    result = result.replace(']', '')
    result = result.replace('^', '')
    result = re.sub('ㅠ|ㅜ|ㅎ|ㅋ|ㅇ|!|~', '', result)
    
    #        정규표현식
    result = re.compile("태그\s디렉터리.+").sub("", result)
    result = re.compile("[a-zA-Z]").sub("", result)
    result_list.append(result)

result_list

['프로미스나인 사진 이쁘거주세요프로미스나인사진 이쁘거 사진주세요',
 '프로미스나인안녕하세요 입덕 11일차 플로버입니다. 프로미스나인에 대해서 알고싶은데요.1. 프로미스나인이 콘서트를 했었나요2. 프로미스나인 수록곡 추천해주세요3.프로미스나인 사진 주세요(배경화면 할수있는 단체사진이요)대답해주시는분 넘 감사하고 사랑해요♡',
 '프로미스나인 맴버별 사진지식인님 안녕하세요답변을 넘 잘해주셔서 1대1 합니다제가 프로미스나인 입덕 위기여가지고 맴버별로 사진 부탁드려요예:이새롬 하고 사진, 송하영 하고 사진 이렇게 나눠서 사진 부탁드립니다 맴버당 20장 이상씩 주세욥그리고 셀카 위주로 부탁드립니다',
 '프로미스나인 채영 머리 무슨 컷인가요 옆에 저렇게 층처럼 나뉜 머리。。。',
 '프로미스나인 서연이 입은옷 브랜드 예쁜 체크 셔츠 어디껀가요사진은 프로미스나인 채널나인에서 캡쳐했어요',
 '프로미스나인 입덕 본진이 사라져서 프로미스나인 노래가 좋아서 최근에 무대 영상 찾아보고 있거든요 근데 아직 멤버 얼굴 구별이 조금씩 힘들어용1. 아이돌학교 순위2. 프로미스나인 팬 디스코드3. 프로미스나인 방주4. 출연 예능 영상5. 프로미스나인 단독 영상6. 리얼리티7. 뭐부터 봐야되는지8. 멤버 얼굴 사진,구별법에 답해주세용',
 '프로미스나인 사진이 사진 좀 구할수있을까요 ',
 '프로미스나인 이채영 사진아래 사진 프로미스나인 이채영님이신데 이사진 고화질로 원본 있으신가요 있으시면 보내주세요',
 '30)) 프로미스나인 이서연님 사진 원본좀 찾아주세요 인스타 돌아다니다가 봤는데 프로미스나인 이서연님 인것 같더라구요..예뻐서 저장하고싶은데 원본, 출처좀 부탁드립니다 ',
 '90프로미스나인 사진 출처가 알고싶어요... 꼭여름앨범 개인컷 같은데 어디서 얻는지 출처가 어딘지 꼭 알고싶네요 너무예뻐서.. 배경화면으로 쓰려구요아시는분 공유해주시면 감사하겠습니다이왕이면 고화질로다가요',
 '프로미스나인 사진프로미스나인 입덕해서요  사진좀 주실수 있을가욘 ',
 '프로미스나인 박지원 사

In [ ]:
data = {'비투비' : result_list}
df= pd.DataFrame(data)
df

,프로미스나인
0,프로미스나인 사진 이쁘거주세요프로미스나인사진 이쁘거 사진주세요
1,프로미스나인안녕하세요 입덕 11일차 플로버입니다. 프로미스나인에 대해서 알고싶은데요...
2,프로미스나인 맴버별 사진지식인님 안녕하세요답변을 넘 잘해주셔서 1대1 합니다제가 프...
3,프로미스나인 채영 머리 무슨 컷인가요 옆에 저렇게 층처럼 나뉜 머리。。。
4,프로미스나인 서연이 입은옷 브랜드 예쁜 체크 셔츠 어디껀가요사진은 프로미스나인 채널...
...,...
901,예쁜 여돌 고화질좀요제발요
902,"프로듀스48 연습생들 사진 부탁드려요(장원영, 안유진, 사쿠라, 권은비, 최예나, ..."
903,예쁜 여자아이돌 파고싶어요 진짜 엄청 노래잘하거나 얼굴이쁘면 좋겠어요 그래서 마마무...
904,영화 추천 점안녕하세요. 영화 추천 부탁합니다 1.액션.스릴.판타지.코미디. 추천 ...


In [ ]:
df.to_csv('비투비.csv', encoding='utf-8-sig')